In [1]:
from collections import defaultdict
import json
from pathlib import Path
import sys

import pandas as pd
from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
import numpy as np
import torch
from torch.utils.data import DataLoader
import wandb

sys.path.insert(0, '../agenticadmet')
from datasets import RegressionDataset
from eval import eval_admet, extract_preds, extract_refs
from models import TransformerRegressionModel
from utils import CheckpointParams

[03:00:52] Initializing Normalizer


In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
NUM_WORKERS = 0 # number of workers for dataloader. 0 means using main process for data loading
SMILES_COLUMN = 'smiles_std'
TARGET_COLUMNS = ['LogHLM', 'LogMLM', 'LogD', 'LogKSOL', 'LogMDR1-MDCKII']

MODEL_PARAMS = {
    'config': {
        'vocab_size': 500,
        'hidden_size': 384,
        'num_hidden_layers': 6,
        'num_attention_heads': 8,
        'intermediate_size': 1024,
        'hidden_act': "gelu",
        'hidden_dropout_prob': 0.1,
        'attention_probs_dropout_prob': 0.1,
        'max_position_embeddings': 512,
        'initializer_range': 0.02,
        'layer_norm_eps': 1e-12,
        'pad_token_id': 0,
        'position_embedding_type': "absolute",
        'use_cache': True,
        'type_vocab_size': 2
    },
    'output_dim': len(TARGET_COLUMNS),
    'bias_final': False
}
MODEL_CONFIG = {
    "hidden_dim": 384,
    "num_layers": 1,
    "dropout": 0.034136,
    "weight_decay": 0.000810,
    "batch_size": 32,
    "max_epochs": 200
}
TOKENIZER_NAME = '<gs_bucket>/artifacts/tokenizers/zinc'
CHECKPOINTS = [
    CheckpointParams(
        path=str(Path('../output/artifacts/mol_mlm_roberta_zinc/last.ckpt').absolute()),
        module_from='roberta',
        module_to='roberta',
        strict=True
    )
]

In [4]:
def prepare_data(input_path, batch_size):
    train_dset = RegressionDataset(
        data_path=input_path,
        smiles_col=SMILES_COLUMN,
        target_cols=TARGET_COLUMNS,
        split='train',
        tokenizer_name=TOKENIZER_NAME,
        mol_masking_prob=0.3,
        mol_masking_val=0.15
    )
    val_dset = RegressionDataset(
        data_path=input_path,
        smiles_col=SMILES_COLUMN,
        target_cols=TARGET_COLUMNS,
        split='val',
        tokenizer_name=TOKENIZER_NAME
    )
    pred_dset = RegressionDataset(
        data_path=input_path,
        smiles_col=SMILES_COLUMN,
        target_cols=TARGET_COLUMNS,
        split=None,
        tokenizer_name=TOKENIZER_NAME,
        remove_all_nan_targets=False
    )

    train_loader = DataLoader(
        train_dset, batch_size=batch_size, shuffle=True,
        num_workers=NUM_WORKERS, collate_fn=train_dset.collate_fn
    )
    val_loader = DataLoader(
        val_dset, batch_size=batch_size, shuffle=False,
        num_workers=NUM_WORKERS, collate_fn=val_dset.collate_fn
    )
    pred_loader = DataLoader(
        pred_dset, batch_size=batch_size, shuffle=False,
        num_workers=NUM_WORKERS, collate_fn=pred_dset.collate_fn
    )

    return train_loader, val_loader, pred_loader

In [5]:
def train_model(config, train_loader, val_loader, save_dir, run_idx):
    model_params = MODEL_PARAMS.copy()
    model_params['hidden_dim'] = config['hidden_dim']
    model_params['num_layers'] = config['num_layers']
    model_params['dropout'] = config['dropout']
    weight_decay = config['weight_decay']

    model = TransformerRegressionModel(
        model_name='roberta-base',
        model_params=model_params,
        weight_decay=weight_decay,
        checkpoints=CHECKPOINTS
    )

    ckpt_callback = ModelCheckpoint(
        save_top_k=0,
        save_last=True
    )

    exp_name = f"roberta_run_{run_idx}"
    logger = WandbLogger(
        project="admet-challenge",
        name=exp_name,
        prefix=f"{save_dir.stem}",
        save_dir=f"../wandb/{exp_name}"
    )

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=config['max_epochs'],
        enable_progress_bar=False,
        callbacks=[ckpt_callback],
        default_root_dir=save_dir,
        logger=logger
    )

    try:
        trainer.fit(model, train_loader, val_loader)
    except Exception as e:
        logger.finalize("failed")
        wandb.finish(exit_code=1)
        raise e
    else:
        logger.finalize("success")

    return model

def predict(model, pred_loader):
    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        enable_progress_bar=False
    )

    model.eval()
    preds = trainer.predict(model, pred_loader, return_predictions=True)
    preds = torch.cat(preds)

    return preds

In [6]:
def train_and_eval(input_paths, save_dirs, run_idx):
    for input_path, save_dir in zip(input_paths, save_dirs):
        print(f"Training and predicting on {input_path}")
        train_loader, val_loader, pred_loader = prepare_data(input_path, MODEL_CONFIG['batch_size'])
        model = train_model(MODEL_CONFIG, train_loader, val_loader, save_dir, run_idx)
        preds = predict(model, pred_loader)

        input_df = pred_loader.dataset.data.copy()
        output_df = input_df.copy()
        output_df[["pred_" + t for t in TARGET_COLUMNS]] = preds
        save_dir.mkdir(parents=True, exist_ok=True)
        output_df.to_csv(save_dir / "predictions.csv", index=False)

        train_preds = extract_preds(output_df[input_df["split"] == "train"])
        train_refs = extract_refs(input_df[input_df["split"] == "train"])
        val_preds = extract_preds(output_df[input_df["split"] == "val"])
        val_refs = extract_refs(input_df[input_df["split"] == "val"])

        metrics = eval_admet(train_preds, train_refs)
        print("Train metrics:")
        print(json.dumps(metrics, indent=2))

        metrics = eval_admet(val_preds, val_refs)
        print("\nVal metrics:")
        print(json.dumps(metrics, indent=2))
    
    wandb.finish(exit_code=0)

## Run 0

In [7]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/roberta/run_0/split_{k}') for k in range(5)]
RUN_IDX = 0

In [8]:
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../data/asap/datasets/rnd_splits/split_0.csv
0 out of 323 rows are removed due to missing values
0 out of 81 rows are removed due to missing values


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vladvin111 (vladvin-org) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Path ../wandb/roberta_run_0/wandb/ wasn't writable, using system temp directory.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'tra

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU 

Train metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.1164171334834601,
    "r2": 0.9419397233682875
  },
  "HLM": {
    "mean_absolute_error": 0.1028978622056051,
    "r2": 0.9509841057503561
  },
  "LogD": {
    "mean_absolute_error": 0.15070896101436873,
    "r2": 0.9772361119643264
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.07961315075839501,
    "r2": 0.9227259289622288
  },
  "MLM": {
    "mean_absolute_error": 0.08248505188120814,
    "r2": 0.9718402954710741
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.10642443186860742,
    "macro_r2": 0.9529452331032544
  }
}

Val metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.2867313222443941,
    "r2": 0.7013819267458912
  },
  "HLM": {
    "mean_absolute_error": 0.3081166543311521,
    "r2": 0.3878684762909246
  },
  "LogD": {
    "mean_absolute_error": 0.5451459044516087,
    "r2": 0.5842822482942774
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.19047527476813583,
    "r2": 0.5332327900343475
  },
  

split_0-epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇███
split_0-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_epoch,█▅▃▃▃▂▂▂▂▂▂▂▁▂▂▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▅▆▃▃▄▂▂▃▃▂▂▃▃▂▂▃▂▃▂▂▃▃▂▂▂▁▂▃▂▂▂▁▁▂▁▂▁▁▁
split_0-val/mae,▇█▅▅▃▁▂▁▄▃▄▃▃▃▃▂▃▃▃▂▂▃▃▃▃▃▃▂▂▃▂▂▄▃▃▃▂▂▂▂
split_0-val/r2,▁▂▅▆▆▇▇▇█▅▆▅▆▇▆▆▆▆▇▅▆▆▆▆▆▇▆█▇▇▇▆▇▆▆▆▆▅▇▆
split_0-val_loss,██▂▂▃▃▂▄▆▃▃▃▃▃▄▂▃▄▄▃▃▃▄▃▃▄▃▁▂▃▃▄▃▄▄▅▄▅▂▄
split_1-epoch,▁▁▁▁▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
split_1-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_epoch,█▄▃▂▃▂▂▂▂▂▁▁▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,█▆▄▃▄▂▃▂▃▄▂▂▃▃▃▃▂▂▂▂▁▂▁▁▃▁▂▂▁▂▁▂▁▁▂▃▁▁▂▃


## Cleaning up + run 1

In [9]:
def clean_data(input_paths, save_dirs, output_dir, remove_worst_pct):
    smiles_to_remove = defaultdict(set)

    for input_path, save_dir in zip(input_paths, save_dirs):
        input_df = pd.read_csv(input_path)
        input_val_df = input_df[input_df["split"] == "val"]
        output_df = pd.read_csv(save_dir / "predictions.csv")
        output_val_df = output_df[input_df["split"] == "val"]

        for t in TARGET_COLUMNS:
            # Sort by absolute error
            notna_mask = input_val_df[t].notna()
            input_val_df = input_val_df[notna_mask]
            output_val_df = output_val_df[notna_mask]

            mae = np.abs(input_val_df[t] - output_val_df[f"pred_{t}"])
            sorted_idx = np.argsort(mae)[::-1]
            smiles_to_remove[t].update(
                input_val_df.iloc[sorted_idx[:int(remove_worst_pct * len(sorted_idx))]]["cxsmiles_std"].tolist()
            )

    for input_path in input_paths:
        input_df = pd.read_csv(input_path)
        for t in TARGET_COLUMNS:
            input_df.loc[input_df["cxsmiles_std"].isin(smiles_to_remove[t]) & (input_df["split"] == "train"), t] = np.nan

        input_df.to_csv(output_dir / input_path.name, index=False)

In [10]:
output_dir = Path("../output/asap/rnd_splits/roberta/run_0/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [11]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [12]:
pd.read_csv(input_paths[0])[TARGET_COLUMNS].isna().sum()

LogHLM            150
LogMLM            140
LogD               86
LogKSOL            74
LogMDR1-MDCKII     15
dtype: int64

In [13]:
pd.read_csv(output_dir / input_paths[0].name)[TARGET_COLUMNS].isna().sum()

LogHLM            189
LogMLM            173
LogD              116
LogKSOL            99
LogMDR1-MDCKII     40
dtype: int64

In [14]:
input_paths = [Path(f'../output/asap/rnd_splits/roberta/run_0/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/roberta/run_1/split_{k}') for k in range(5)]
RUN_IDX = "1_clean_worst_pct_0.2"

In [15]:
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../output/asap/rnd_splits/roberta/run_0/cleaned/split_0.csv
1 out of 323 rows are removed due to missing values
0 out of 81 rows are removed due to missing values


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/roberta_run_1_clean_worst_pct_0.2/wandb/ wasn't writable, using system temp directory.


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'tra

Train metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.1861880703660957,
    "r2": 0.8391420174084648
  },
  "HLM": {
    "mean_absolute_error": 0.1908893015533692,
    "r2": 0.6894910349779328
  },
  "LogD": {
    "mean_absolute_error": 0.18807809859908095,
    "r2": 0.9608093818258516
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.12862705999906573,
    "r2": 0.775925789907343
  },
  "MLM": {
    "mean_absolute_error": 0.2190807342436571,
    "r2": 0.6857194162123077
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.18257265295225372,
    "macro_r2": 0.7902175280663799
  }
}

Val metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.35872086094375744,
    "r2": 0.4466622297405026
  },
  "HLM": {
    "mean_absolute_error": 0.33513700619755515,
    "r2": 0.3943504962569053
  },
  "LogD": {
    "mean_absolute_error": 0.5637464449444756,
    "r2": 0.6422735980639326
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.251403870991386,
    "r2": 0.40430013468538895
  },
  "

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU 

Train metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.20594558012252492,
    "r2": 0.7471845358715377
  },
  "HLM": {
    "mean_absolute_error": 0.1983428200696025,
    "r2": 0.6458137340257575
  },
  "LogD": {
    "mean_absolute_error": 0.17462980678468418,
    "r2": 0.9682475988525927
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.09449591768654753,
    "r2": 0.8353163475080205
  },
  "MLM": {
    "mean_absolute_error": 0.19677022944513534,
    "r2": 0.6361286880435564
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.17403687082169889,
    "macro_r2": 0.766538180860293
  }
}

Val metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.26153112658020516,
    "r2": 0.773941659086051
  },
  "HLM": {
    "mean_absolute_error": 0.27148158146499257,
    "r2": 0.5707034832706072
  },
  "LogD": {
    "mean_absolute_error": 0.5877801813234885,
    "r2": 0.5535538397222723
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.19545590712410288,
    "r2": 0.505989661283273
  },
  

split_0-epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
split_0-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_epoch,█▆▄▂▂▃▂▂▂▂▂▂▁▂▂▁▁▁▂▂▁▁▂▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁
split_0-train_loss_step,▇█▄▃▃▄▄▃▃▃▃▃▃▃▃▃▃▂▄▂▃▂▂▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▁
split_0-val/mae,█▅▅▃▁▂▅▂▂▃▂▁▂▃▂▃▃▅▂▂▂▃▄▃▄▂▄▂▂▃▂▁▂▁▁▂▁▁▂▄
split_0-val/r2,▁▃▃▃▃▃▆▄▆▆▆▆▆▇▅▆▆▆▇▂▅▆▇▆▆▅▆▆▆▇▇▆██▇▇█▇█▇
split_0-val_loss,█▅▄▃▃▃▂▂▂▃▂▂▂▂▃▁▁▂▁▂▃▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▂▂▂▁
split_1-epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇███
split_1-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_epoch,█▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁
split_1-train_loss_step,█▆▇▅▆▃▃▃▃▃▁▁▃▃▂▃▃▃▂▄▂▁▂▂▂▁▂▂▁▂▁▁▄▁▃▁▁▂▁▁


## Cleaning up + run 2

In [16]:
output_dir = Path("../output/asap/rnd_splits/roberta/run_1/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [17]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [18]:
input_paths = [Path(f'../output/asap/rnd_splits/roberta/run_1/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/roberta/run_2/split_{k}') for k in range(5)]
RUN_IDX = "2_clean_worst_pct_0.2"

In [19]:
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../output/asap/rnd_splits/roberta/run_1/cleaned/split_0.csv
3 out of 323 rows are removed due to missing values
0 out of 81 rows are removed due to missing values


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/roberta_run_2_clean_worst_pct_0.2/wandb/ wasn't writable, using system temp directory.


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'tra

Train metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.1688764883476707,
    "r2": 0.810708980990781
  },
  "HLM": {
    "mean_absolute_error": 0.21892033010725034,
    "r2": 0.6044276024825277
  },
  "LogD": {
    "mean_absolute_error": 0.16438498869750173,
    "r2": 0.9726207727307689
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.0830587430861154,
    "r2": 0.8390308523204506
  },
  "MLM": {
    "mean_absolute_error": 0.21806808120499632,
    "r2": 0.6105358902473206
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.17066172628870688,
    "macro_r2": 0.7674648197543699
  }
}

Val metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.31923898589557,
    "r2": 0.490842814061037
  },
  "HLM": {
    "mean_absolute_error": 0.41090528139300986,
    "r2": 0.05414296687545972
  },
  "LogD": {
    "mean_absolute_error": 0.5605308732927822,
    "r2": 0.5977805934594193
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.24737459971764894,
    "r2": 0.27236219014480856
  },
  "

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU 

Train metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.24681134481419628,
    "r2": 0.6644760513740046
  },
  "HLM": {
    "mean_absolute_error": 0.283133056233829,
    "r2": 0.4793869697721731
  },
  "LogD": {
    "mean_absolute_error": 0.2369130607718645,
    "r2": 0.9410693538636942
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.15792643543511825,
    "r2": 0.6391591138866446
  },
  "MLM": {
    "mean_absolute_error": 0.24836378005059304,
    "r2": 0.6029419918234791
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.23462953546112023,
    "macro_r2": 0.665406696143999
  }
}

Val metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.270355116785023,
    "r2": 0.74775974093329
  },
  "HLM": {
    "mean_absolute_error": 0.29268326593558414,
    "r2": 0.45909528491605134
  },
  "LogD": {
    "mean_absolute_error": 0.6434580408831436,
    "r2": 0.48287741382700056
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.26575679788468826,
    "r2": 0.2496315833653031
  },
  "M

split_0-epoch,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇███
split_0-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_epoch,█▆▆▄▆▄▅▂▄▃▃▃▃▃▃▂▂▂▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▂▁▁▂
split_0-train_loss_step,█▅▃▃▂▂▂▃▃▂▂▃▂▂▂▂▂▂▂▂▁▂▁▁▁▂▂▁▁▂▁▁▁▁▁▁▂▁▁▂
split_0-val/mae,█▄▃▃▃▂▂▂▂▃▂▂▂▂▃▂▂▂▃▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▂▂▂▂▂▁
split_0-val/r2,▁▄▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇█▇▇▇█▇██████▇▇
split_0-val_loss,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▁▂▁▁▁▁▁▁
split_1-epoch,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▆▆▇▇▇▇███
split_1-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_epoch,█▄▃▃▃▃▃▂▂▂▂▂▃▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_step,▇▆▆▅█▆▆▄▄▃▃▃▃▄▃▃▃▄▃▃▂▂▂▃▂▂▃▃▃▂▃▃▂▃▂▂▂▂▂▁


## Cleaning up stero impure + run 1

In [20]:
def clean_data(input_paths, save_dirs, output_dir, remove_worst_pct):
    smiles_to_remove = defaultdict(set)

    for input_path, save_dir in zip(input_paths, save_dirs):
        input_df = pd.read_csv(input_path)
        input_val_df = input_df[input_df["split"] == "val"]
        output_df = pd.read_csv(save_dir / "predictions.csv")
        output_val_df = output_df[input_df["split"] == "val"]

        for t in TARGET_COLUMNS:
            # Sort by absolute error
            notna_mask = input_val_df[t].notna()
            input_val_df = input_val_df[notna_mask]
            output_val_df = output_val_df[notna_mask]

            mae = np.abs(input_val_df[t] - output_val_df[f"pred_{t}"])
            sorted_idx = np.argsort(mae)[::-1]
            smiles_to_remove[t].update(
                input_val_df.iloc[sorted_idx[:int(remove_worst_pct * len(sorted_idx))]]["cxsmiles_std"].tolist()
            )

    for input_path in input_paths:
        input_df = pd.read_csv(input_path)
        for t in TARGET_COLUMNS:
            input_df.loc[
                input_df["cxsmiles_std"].isin(smiles_to_remove[t]) & \
                    ~input_df["smiles_ext"].isna() & \
                    (input_df["split"] == "train"),
                t
            ] = np.nan

        input_df.to_csv(output_dir / input_path.name, index=False)

In [21]:
output_dir = Path("../output/asap/rnd_splits/roberta/run_0/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

In [22]:
clean_data(input_paths, save_dirs, output_dir, remove_worst_pct = 0.2)

In [23]:
pd.read_csv(input_paths[0])[TARGET_COLUMNS].isna().sum()

LogHLM            201
LogMLM            186
LogD              132
LogKSOL           109
LogMDR1-MDCKII     55
dtype: int64

In [24]:
tmp = pd.read_csv(input_paths[0])
tmp = tmp[tmp["smiles_ext"].isna()]
tmp[TARGET_COLUMNS].isna().sum()

LogHLM            100
LogMLM             82
LogD               74
LogKSOL            56
LogMDR1-MDCKII     33
dtype: int64

In [25]:
pd.read_csv(output_dir / input_paths[0].name)[TARGET_COLUMNS].isna().sum()

LogHLM            204
LogMLM            188
LogD              134
LogKSOL           109
LogMDR1-MDCKII     56
dtype: int64

In [26]:
tmp = pd.read_csv(output_dir / input_paths[0].name)
tmp = tmp[tmp["smiles_ext"].isna()]
tmp[TARGET_COLUMNS].isna().sum()

LogHLM            100
LogMLM             82
LogD               74
LogKSOL            56
LogMDR1-MDCKII     33
dtype: int64

In [27]:
input_paths = [Path(f'../output/asap/rnd_splits/roberta/run_0/cleaned/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/roberta/run_1/split_{k}') for k in range(5)]
RUN_IDX = "1_clean_worst_pct_0.2_stereo_impure"

In [28]:
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../output/asap/rnd_splits/roberta/run_0/cleaned/split_0.csv
3 out of 323 rows are removed due to missing values
0 out of 81 rows are removed due to missing values


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/roberta_run_1_clean_worst_pct_0.2_stereo_impure/wandb/ wasn't writable, using system temp directory.


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'tra

Train metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.21637993941611766,
    "r2": 0.7582770243124185
  },
  "HLM": {
    "mean_absolute_error": 0.20689065525625036,
    "r2": 0.6510633189776583
  },
  "LogD": {
    "mean_absolute_error": 0.21599684359639457,
    "r2": 0.956354620210111
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.1188643528206647,
    "r2": 0.7913687607928919
  },
  "MLM": {
    "mean_absolute_error": 0.22602909318495562,
    "r2": 0.6126703803350781
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.19683217685487658,
    "macro_r2": 0.7539468209256316
  }
}

Val metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.3663063745356479,
    "r2": 0.3734249731387834
  },
  "HLM": {
    "mean_absolute_error": 0.3511783545996734,
    "r2": 0.22844983126282514
  },
  "LogD": {
    "mean_absolute_error": 0.6061596973655653,
    "r2": 0.5775076071696578
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.24213701043718402,
    "r2": 0.315796488015866
  },
  

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU 

Train metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.22414353032524362,
    "r2": 0.6547209535755518
  },
  "HLM": {
    "mean_absolute_error": 0.2620421606547088,
    "r2": 0.5080727833299641
  },
  "LogD": {
    "mean_absolute_error": 0.1727487322013333,
    "r2": 0.9674805799240126
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.13689891973463994,
    "r2": 0.6549811281113724
  },
  "MLM": {
    "mean_absolute_error": 0.27058520916549894,
    "r2": 0.5662023008892434
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.21328371041628494,
    "macro_r2": 0.6702915491660288
  }
}

Val metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.25754414585017843,
    "r2": 0.7548105852999295
  },
  "HLM": {
    "mean_absolute_error": 0.2608813230939951,
    "r2": 0.5939974777296552
  },
  "LogD": {
    "mean_absolute_error": 0.587562834451596,
    "r2": 0.5146416477669302
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.2848023897378914,
    "r2": -0.01366880114162905
  },
 

split_0-epoch,▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇█
split_0-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_epoch,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_step,█▅▄▃▃▄▃▃▂▂▂▃▂▂▂▂▂▂▂▁▂▁▂▂▁▁▂▁▁▁▂▂▁▁▂▂▁▂▂▂
split_0-val/mae,█▄▄▂▂▂▂▂▂▁▂▂▁▂▁▁▁▁▁▂▂▂▂▂▁▁▂▁▁▁▂▁▁▂▂▂▁▂▁▂
split_0-val/r2,▁▄▅▇██▇▇█████▇▇████▇▇▇▇▇▇██▇▇▇▇█▇██████▇
split_0-val_loss,▅▅▅█▁▄▅▅▃▃▁▄▃▇▅▄▅▃▅▄▄▄▆▅▆▆▇▅▆▄▆▄▃▃▃▄▃▄▆▄
split_1-epoch,▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇██
split_1-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_epoch,█▇▄▃▃▃▃▃▂▃▂▄▂▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▂▁▁▂▁▁▁▁
split_1-train_loss_step,█▄▇▅▄▄▂▃▂▃▂▃▄▂▂▂▂▃▂▂▂▂▃▂▂▃▁▂▂▃▁▁▁▂▂▁▃▂▁▂


## Removing all stereo impure + run 0

In [29]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/split_{k}.csv') for k in range(5)]
output_dir = Path("../data/asap/datasets/rnd_splits/stereo_pure")
output_dir.mkdir(parents=True, exist_ok=True)

for input_path in input_paths:
    input_df = pd.read_csv(input_path)
    input_df = pd.concat([
        input_df[(input_df["split"] == "train") & input_df["smiles_ext"].isna()],
        input_df[input_df["split"] == "val"]
    ])
    input_df.to_csv(output_dir / input_path.name, index=False)

In [30]:
input_paths = [Path(f'../data/asap/datasets/rnd_splits/stereo_pure/split_{k}.csv') for k in range(5)]
save_dirs = [Path(f'../output/asap/rnd_splits/roberta/run_0/stereo_pure/split_{k}') for k in range(5)]
RUN_IDX = "0_stereo_pure"
train_and_eval(input_paths, save_dirs, RUN_IDX)

Training and predicting on ../data/asap/datasets/rnd_splits/stereo_pure/split_0.csv
0 out of 166 rows are removed due to missing values
0 out of 81 rows are removed due to missing values


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: WARNING Path ../wandb/roberta_run_0_stereo_pure/wandb/ wasn't writable, using system temp directory.


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'tra

Train metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.20010931505162605,
    "r2": 0.8915152256344944
  },
  "HLM": {
    "mean_absolute_error": 0.09812466124933268,
    "r2": 0.9636607789535374
  },
  "LogD": {
    "mean_absolute_error": 0.13463153686431736,
    "r2": 0.982303189357938
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.08364396180401804,
    "r2": 0.919976012823846
  },
  "MLM": {
    "mean_absolute_error": 0.10709459299818337,
    "r2": 0.9571038976882112
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.12472081359349549,
    "macro_r2": 0.9429118208916055
  }
}

Val metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.5393368077844162,
    "r2": -0.18617652139218221
  },
  "HLM": {
    "mean_absolute_error": 0.38943063077011925,
    "r2": 0.037898266646701595
  },
  "LogD": {
    "mean_absolute_error": 0.7456228902984838,
    "r2": 0.4414945104236615
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.34159407699615496,
    "r2": -0.06403151123248496


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type         | Params | Mode 
---------------------------------------------------
0 | roberta   | RobertaModel | 8.7 M  | train
1 | predictor | MLP          | 149 K  | train
2 | criterion | MSE          | 0      | train
3 | metrics   | ModuleList   | 0      | train
---------------------------------------------------
8.8 M     Trainable params
0         Non-trainable params
8.8 M     Total params
35.297    Total estimated model params size (MB)
128       Modules in train mode
0         Modules in eval mode


Getting checkpoint from /home/jupyter/AgenticADMET/notebooks/../output/artifacts/mol_mlm_roberta_zinc/last.ckpt...
Loading checkpoint from roberta to roberta...
<All keys matched successfully>


/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/opt/conda/envs/admet/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (cuda), used: True
TPU a

Train metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.10881194538078122,
    "r2": 0.9559973147700999
  },
  "HLM": {
    "mean_absolute_error": 0.13697251945789846,
    "r2": 0.9315443100159168
  },
  "LogD": {
    "mean_absolute_error": 0.13320418093786685,
    "r2": 0.9806549379758814
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.0875027711120591,
    "r2": 0.8999674472779172
  },
  "MLM": {
    "mean_absolute_error": 0.08088506654617339,
    "r2": 0.9752515762132834
  },
  "aggregated": {
    "macro_mean_absolute_error": 0.1094752966869558,
    "macro_r2": 0.9486831172506196
  }
}

Val metrics:
{
  "KSOL": {
    "mean_absolute_error": 0.4163545562125055,
    "r2": 0.4205903388125979
  },
  "HLM": {
    "mean_absolute_error": 0.450075913200766,
    "r2": -0.10658784070746163
  },
  "LogD": {
    "mean_absolute_error": 0.8097103596528371,
    "r2": 0.2681078579989802
  },
  "MDR1-MDCKII": {
    "mean_absolute_error": 0.29141314974787097,
    "r2": 0.12345889966982182
  },


split_0-epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇█
split_0-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_0-train_loss_epoch,█▅▄▄▃▂▂▂▂▂▁▂▂▂▂▂▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁
split_0-train_loss_step,█▃▃▃▂▃▂▁▁▂▂▁▂▂▁▁▁▁▁▂▁▁▁▁
split_0-val/mae,█▆▅▆▄▃▄▄▂▃▃▃▁▂▂▂▃▂▂▃▂▅▃▃▅▄▃▃▃▃▄▄▅▅▂▃▂▂▄▁
split_0-val/r2,▃▁▅▅▄█▆▅▅▄▆▅▆▇▆▇▇▇▆▅▂▅▂▅▃▅▆▅▅▅▅▅▃▄▆▅▆▆█▅
split_0-val_loss,▅▄▇▆▇▃▅▅▂▄▅▅▅▃▄▄▆▄▅▆█▇▅█▆▆▅▄▅▅▆█▄▄▄▆▁▄▂▂
split_1-epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
split_1-lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
split_1-train_loss_epoch,█▇▇▅▃▃▃▄▃▃▃▂▂▂▂▂▂▂▃▂▁▂▁▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂
split_1-train_loss_step,█▃▁▄▃▂▂▂▁▃▂▇▁▁▂▂▁▁▁▂▁▁▁▃
